## Installing required packages

In [8]:
!pip install pyspellchecker 
!pip install nltk


## Importing required libraries

In [9]:
from spellchecker import SpellChecker
import tkinter as tk
from tkinter import scrolledtext, messagebox
import re
import time
from collections import Counter
import nltk
nltk.download('punkt', quiet=True)


True

## creating main autocorrect engine

In [12]:
class AutocorrectEngine:
    def __init__(self):
        self.spell = SpellChecker()
        self.custom_words = set()

    def add_custom_words(self, words):
        """Add custom words to the dictionary"""
        self.custom_words.update(words)
        self.spell.word_frequency.load_words(words)

    def identify_misspelled_words(self, text):
        """Identify misspelled words in the text"""
        words = re.findall(r'\b[a-zA-Z]+\b', text.lower())
        misspelled = self.spell.unknown(words)
        return list(misspelled)

    def get_corrections(self, word):
        """Get correction suggestions for a misspelled word"""
        candidates = self.spell.candidates(word)
        if candidates:
            return list(candidates)[:5]  # Return top 5 suggestions
        return []
    
    def autocorrect_text(self, text, auto_correct=True):
        """Autocorrect the entire text"""
        words = text.split()
        corrected_words = []
        corrections_made = {}
        
        for word in words:
            clean_word = re.sub(r'[^\w]', '', word.lower())
            
            if clean_word and clean_word in self.spell.unknown([clean_word]):
                if auto_correct:
                    correction = self.spell.correction(clean_word)
                    if correction and correction != clean_word:
                        corrected_word = self.preserve_formatting(word, correction)
                        corrected_words.append(corrected_word)
                        corrections_made[word] = corrected_word
                    else:
                        corrected_words.append(word)
                else:
                    corrected_words.append(word)
            else:
                corrected_words.append(word)
        
        return ' '.join(corrected_words), corrections_made
        
    def preserve_formatting(self, original, correction):
        """Preserve capitalization and punctuation from original word"""
        if original.isupper():
            return correction.upper()
        elif original.istitle():
            return correction.capitalize()
        return correction

## creating function for real time spell checking

In [13]:
class RealTimeChecker:
    def __init__(self, autocorrect_engine):
        self.engine = autocorrect_engine
        self.last_check_time = 0
        self.check_delay = 0.5

    def check_word_as_typed(self, word):
        """Check a single word as it's being typed"""
        clean_word = re.sub(r'[^\w]', '', word.lower())
        if len(clean_word) > 2:
            if clean_word in self.engine.spell.unknown([clean_word]):
                suggestions = self.engine.get_corrections(clean_word)
                return {
                    'is_misspelled': True,
                    'word': word,
                    'suggestions': suggestions
                }
        return {
            'is_misspelled': False,
            'word': word,
            'suggestions': []
        }

    def analyze_text_statistics(self, text):
        """Provide statistics about the text"""
        words = re.findall(r'\b[a-zA-Z]+\b', text.lower())
        misspelled = self.engine.identify_misspelled_words(text)
        
        return {
            'total_words': len(words),
            'misspelled_count': len(misspelled),
            'accuracy_percentage': round((len(words) - len(misspelled)) / len(words) * 100, 2) if words else 100,
            'misspelled_words': misspelled
        }

## Creating a user friendly interface

In [14]:
class AutocorrectKeyboard:
    def __init__(self):
        self.engine = AutocorrectEngine()
        self.checker = RealTimeChecker(self.engine)
        self.root = tk.Tk()
        self.setup_gui()
        
    def setup_gui(self):
        self.root.title("Autocorrect Keyboard System")
        self.root.geometry("800x600")
        
        
        main_frame = tk.Frame(self.root)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        
        title_label = tk.Label(main_frame, text="Autocorrect Keyboard System", 
                              font=("Arial", 16, "bold"))
        title_label.pack(pady=(0, 10))
        
        
        tk.Label(main_frame, text="Type your text below:", font=("Arial", 10)).pack(anchor="w")
        self.text_area = scrolledtext.ScrolledText(main_frame, height=10, wrap=tk.WORD)
        self.text_area.pack(fill=tk.BOTH, expand=True, pady=(5, 10))
        
        
        self.text_area.bind('<KeyRelease>', self.on_text_change)
        
        
        button_frame = tk.Frame(main_frame)
        button_frame.pack(fill=tk.X, pady=(0, 10))
        
        tk.Button(button_frame, text="Check Spelling", command=self.check_spelling,
                 bg="#4CAF50", fg="white", font=("Arial", 10)).pack(side=tk.LEFT, padx=(0, 5))
        
        tk.Button(button_frame, text="Auto Correct", command=self.auto_correct,
                 bg="#2196F3", fg="white", font=("Arial", 10)).pack(side=tk.LEFT, padx=(0, 5))
        
        tk.Button(button_frame, text="Clear Text", command=self.clear_text,
                 bg="#f44336", fg="white", font=("Arial", 10)).pack(side=tk.LEFT, padx=(0, 5))
        
        tk.Button(button_frame, text="Add Words", command=self.add_custom_words,
                 bg="#FF9800", fg="white", font=("Arial", 10)).pack(side=tk.LEFT)
        
        
        tk.Label(main_frame, text="Results:", font=("Arial", 10, "bold")).pack(anchor="w")
        self.results_area = scrolledtext.ScrolledText(main_frame, height=8, wrap=tk.WORD)
        self.results_area.pack(fill=tk.BOTH, expand=True)
        
        
        self.status_var = tk.StringVar()
        self.status_var.set("Ready - Start typing to see real-time suggestions")
        status_bar = tk.Label(main_frame, textvariable=self.status_var, 
                             relief=tk.SUNKEN, anchor="w")
        status_bar.pack(fill=tk.X, pady=(5, 0))
    
    def on_text_change(self, event):
        """Handle real-time text changes"""
        current_time = time.time()
        if current_time - self.checker.last_check_time > self.checker.check_delay:
            text = self.text_area.get("1.0", tk.END).strip()
            if text:
                stats = self.checker.analyze_text_statistics(text)
                self.status_var.set(f"Words: {stats['total_words']} | "
                                  f"Misspelled: {stats['misspelled_count']} | "
                                  f"Accuracy: {stats['accuracy_percentage']}%")
            self.checker.last_check_time = current_time
    
    def check_spelling(self):
        """Check spelling and show detailed results"""
        text = self.text_area.get("1.0", tk.END).strip()
        if not text:
            messagebox.showwarning("Warning", "Please enter some text first!")
            return
        
        misspelled_words = self.engine.identify_misspelled_words(text)
        stats = self.checker.analyze_text_statistics(text)
        
        
        self.results_area.delete("1.0", tk.END)
        
        result_text = f"=== SPELL CHECK RESULTS ===\n\n"
        result_text += f"Total Words: {stats['total_words']}\n"
        result_text += f"Misspelled Words: {stats['misspelled_count']}\n"
        result_text += f"Accuracy: {stats['accuracy_percentage']}%\n\n"
        
        if misspelled_words:
            result_text += "MISSPELLED WORDS AND SUGGESTIONS:\n"
            result_text += "-" * 40 + "\n"
            
            for word in misspelled_words:
                suggestions = self.engine.get_corrections(word)
                result_text += f" '{word}' → Suggestions: {', '.join(suggestions[:3]) if suggestions else 'No suggestions'}\n"
        else:
            result_text += "No spelling errors found! Great job!\n"
        
        self.results_area.insert("1.0", result_text)
    
    def auto_correct(self):
        """Auto-correct the text"""
        text = self.text_area.get("1.0", tk.END).strip()
        if not text:
            messagebox.showwarning("Warning", "Please enter some text first!")
            return
        
        corrected_text, corrections = self.engine.autocorrect_text(text)
        
        
        self.text_area.delete("1.0", tk.END)
        self.text_area.insert("1.0", corrected_text)
        
        
        self.results_area.delete("1.0", tk.END)
        
        if corrections:
            result_text = "=== AUTO-CORRECTION RESULTS ===\n\n"
            result_text += f"Corrections Made: {len(corrections)}\n\n"
            result_text += "CHANGES:\n"
            result_text += "-" * 30 + "\n"
            
            for original, corrected in corrections.items():
                result_text += f"'{original}' → '{corrected}'\n"
        else:
            result_text = "No corrections needed! Text is already correct.\n"
        
        self.results_area.insert("1.0", result_text)
        messagebox.showinfo("Success", f"Auto-correction complete! {len(corrections)} words corrected.")
    
    def clear_text(self):
        """Clear all text areas"""
        self.text_area.delete("1.0", tk.END)
        self.results_area.delete("1.0", tk.END)
        self.status_var.set("Text cleared - Ready for new input")
    
    def add_custom_words(self):
        """Add custom words to dictionary"""
        dialog = CustomWordsDialog(self.root, self.engine)
    
    def run(self):
        """Start the application"""
        self.root.mainloop()

class CustomWordsDialog:
    def __init__(self, parent, engine):
        self.engine = engine
        self.dialog = tk.Toplevel(parent)
        self.dialog.title("Add Custom Words")
        self.dialog.geometry("400x300")
        self.dialog.grab_set()
        
        tk.Label(self.dialog, text="Add custom words (one per line):", 
                font=("Arial", 10)).pack(pady=10)
        
        self.text_entry = scrolledtext.ScrolledText(self.dialog, height=10)
        self.text_entry.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        button_frame = tk.Frame(self.dialog)
        button_frame.pack(pady=10)
        
        tk.Button(button_frame, text="Add Words", command=self.add_words,
                 bg="#4CAF50", fg="white").pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="Cancel", command=self.dialog.destroy,
                 bg="#f44336", fg="white").pack(side=tk.LEFT, padx=5)
    
    def add_words(self):
        words_text = self.text_entry.get("1.0", tk.END).strip()
        if words_text:
            words = [word.strip().lower() for word in words_text.split('\n') if word.strip()]
            self.engine.add_custom_words(words)
            messagebox.showinfo("Success", f"Added {len(words)} custom words to dictionary!")
            self.dialog.destroy()


## Testing and Demo

In [15]:
def demo_autocorrect_system():
    """Demo function to test the autocorrect system without GUI"""
    print("=== AUTOCORRECT KEYBOARD SYSTEM DEMO ===\n")
    
    
    engine = AutocorrectEngine()
    
    
    test_sentences = [
        "Ths is a tets sentance with speling erors.",
        "I am wrting a progam for autocorect functionaliy.",
        "Machin lerning and artifical inteligence are fasinating feilds.",
        "Pythn is a powrful programing languag for beginers."
    ]
    
    for i, sentence in enumerate(test_sentences, 1):
        print(f"Test {i}:")
        print(f"Original: {sentence}")
        
        
        misspelled = engine.identify_misspelled_words(sentence)
        print(f"Misspelled words: {misspelled}")
        
        
        print("Suggestions:")
        for word in misspelled[:3]:  # Show first 3 misspelled words
            suggestions = engine.get_corrections(word)
            print(f"  '{word}' → {suggestions[:3]}")
        
        
        corrected, corrections = engine.autocorrect_text(sentence)
        print(f"Corrected: {corrected}")
        print(f"Changes made: {len(corrections)}")
        print("-" * 50)

# Run the demo
demo_autocorrect_system()


=== AUTOCORRECT KEYBOARD SYSTEM DEMO ===

Test 1:
Original: Ths is a tets sentance with speling erors.
Misspelled words: ['sentance', 'speling', 'ths', 'erors']
Suggestions:
  'sentance' → ['sentence']
  'speling' → ['spewing', 'seeling', 'spelling']
  'ths' → ["t's", 'th', 'this']
Corrected: The is a tets sentence with spelling errors
Changes made: 4
--------------------------------------------------
Test 2:
Original: I am wrting a progam for autocorect functionaliy.
Misspelled words: ['progam', 'wrting', 'autocorect', 'functionaliy']
Suggestions:
  'progam' → ['program']
  'wrting' → ['writing', 'wring']
  'autocorect' → []
Corrected: I am writing a program for autocorect functionality
Changes made: 3
--------------------------------------------------
Test 3:
Original: Machin lerning and artifical inteligence are fasinating feilds.
Misspelled words: ['machin', 'lerning', 'feilds', 'inteligence', 'artifical', 'fasinating']
Suggestions:
  'machin' → ['machine']
  'lerning' → ['learning

In [16]:
if __name__ == "__main__":
    app = AutocorrectKeyboard()
    print("Launching Autocorrect Keyboard System...")
    print("Features:")
    print("- Real-time spell checking")
    print("- Autocorrection suggestions")
    print("- Custom word dictionary")
    print("- Text statistics")
    app.run()


Launching Autocorrect Keyboard System...
Features:
- Real-time spell checking
- Autocorrection suggestions
- Custom word dictionary
- Text statistics


## Context-Aware Corrections

In [17]:
def context_aware_correction(self, sentence, word_position):
    """Improve corrections based on context"""
    words = sentence.split()
    if word_position > 0:
        prev_word = words[word_position - 1]
        


## Learning User Preferences

In [ ]:
class UserLearning:
    def __init__(self):
        self.user_corrections = {}
    
    def learn_correction(self, original, user_choice):
        """Learn from user's correction choices"""
        self.user_corrections[original] = user_choice


In [2]:
# Launch the full GUI application
if __name__ == "__main__":
    app = AutocorrectKeyboard()
    print("Launching Autocorrect Keyboard System...")
    print("Features:")
    print("- Real-time spell checking")
    print("- Autocorrection suggestions") 
    print("- Custom word dictionary")
    print("- Text statistics")
    app.run()


NameError: name 'AutocorrectKeyboard' is not defined